# 1. Import libraries & load data

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import evaluation
import csv

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from hep_ml.gradientboosting import UGradientBoostingClassifier,LogLossFunction
from sklearn.metrics import roc_curve, auc
from __future__ import division

In [3]:
train = pd.read_csv('training.csv', index_col = 'id')
test  = pd.read_csv('test.csv', index_col = 'id')

train = train[train['min_ANNmuon'] > 0.4]

In [4]:
check_agreement = pd.read_csv('check_agreement.csv', index_col='id')

def check_ks(agreement_probs, check_agreement = check_agreement):

    ks = evaluation.compute_ks(
        agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    return 'KS metric', ks, ks < 0.09

check_correlation = pd.read_csv('check_correlation.csv', index_col='id')

# 2. Training features

### 2.1 Create new geometric features

In [5]:
list1 = ['FlightDistance', 'FlightDistanceError', 'LifeTime', 'IP', 'IPSig', 'VertexChi2', 'dira', 'pt',
         'DOCAone', 'DOCAtwo', 'DOCAthree', 'IP_p0p2', 'IP_p1p2', 'isolationa', 'isolationb', 'isolationc',
         'isolationd', 'isolatione', 'isolationf', 'iso', 'CDF1', 'CDF2', 'CDF3', 'ISO_SumBDT',
         'p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT', 'p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof', 
         'p0_IP', 'p0_IPSig', 'p1_IP', 'p1_IPSig', 'p2_IP', 'p2_IPSig',
# Extra features:
         'E',
         'FlightDistanceSig',
         'DOCA_sum',
         'isolation_sum',
         'IsoBDT_sum',
         'track_Chi2Dof',
         'IP_sum',
         'IPSig_sum',
         'CDF_sum']

In [6]:
# Physical constants:
c = 299.792458     # Speed of light
m_mu = 105.6583715 # Muon mass (in MeV)
m_tau = 1776.82    # Tau mass (in MeV)

In [7]:
# some feature engineering work
def add_extra_geometric_features(df):
    df['E0'] = np.sqrt(np.square(m_mu) + np.square(df['p0_p']))
    df['E1'] = np.sqrt(np.square(m_mu) + np.square(df['p1_p']))
    df['E2'] = np.sqrt(np.square(m_mu) + np.square(df['p2_p']))
    df['E'] = df['E0'] + df['E1'] + df['E2']
    df['FlightDistanceSig'] = df['FlightDistance'] / df['FlightDistanceError']
    df['DOCA_sum'] = df['DOCAone'] + df['DOCAtwo'] + df['DOCAthree']
    df['isolation_sum'] = df['isolationa'] + df['isolationb'] + df['isolationc'] + df['isolationd'] + df['isolatione'] + df['isolationf']
    df['IsoBDT_sum'] = df['p0_IsoBDT'] + df['p1_IsoBDT'] + df['p2_IsoBDT']
    df['track_Chi2Dof'] = np.sqrt(np.square(df['p0_track_Chi2Dof'] - 1) + np.square(df['p1_track_Chi2Dof'] - 1) + np.square(df['p2_track_Chi2Dof'] - 1))
    df['IP_sum'] = df['p0_IP'] + df['p1_IP'] + df['p2_IP']
    df['IPSig_sum'] = df['p0_IPSig'] + df['p1_IPSig'] + df['p2_IPSig']
    df['CDF_sum'] = df['CDF1'] + df['CDF2'] + df['CDF3']
    
    return df

In [8]:
# Add extra geometric features:
train_with_extra_geometric_features = add_extra_geometric_features(train)
test_with_extra_geometric_features = add_extra_geometric_features(test)

check_agreement_with_extra_geometric_features = add_extra_geometric_features(check_agreement)
check_correlation_with_extra_geometric_features = add_extra_geometric_features(check_correlation)

### 2.2 Create new kinematic features

In [9]:
list2 = [
# Original features:
         'dira', 'pt', 'p0_pt', 'p0_p', 'p0_eta', 'p1_pt', 'p1_p', 'p1_eta', 'p2_pt', 'p2_p', 'p2_eta',
# Extra features:
         'E',
         'pz',
         'beta',
         'gamma',
         'beta_gamma',
         'Delta_E',
         'Delta_M',
         'flag_M',
         'E0',
         'E1',
         'E2',
         'E0_ratio',
         'E1_ratio',
         'E2_ratio',
         'p0_pt_ratio',
         'p1_pt_ratio',
         'p2_pt_ratio',
         'eta_01',
         'eta_02',
         'eta_12',
         't_coll'
         ]

In [10]:
def add_extra_kinematic_features(df):
    df['E0'] = np.sqrt(np.square(m_mu) + np.square(df['p0_p']))
    df['E1'] = np.sqrt(np.square(m_mu) + np.square(df['p1_p']))
    df['E2'] = np.sqrt(np.square(m_mu) + np.square(df['p2_p']))
    df['E'] = df['E0'] + df['E1'] + df['E2']
    df['pz'] = df['p0_pt'] * np.sinh(df['p0_eta']) + df['p1_pt'] * np.sinh(df['p1_eta']) + df['p2_pt'] * np.sinh(df['p2_eta'])
    df['gamma'] = df['E'] / np.sqrt(np.square(df['E']) - np.square(np.sqrt(np.square(df['pt']) + np.square(df['pz']))))
    df['beta'] = np.sqrt(np.square(df['gamma']) - 1.) / df['gamma']
    df['beta_gamma'] = df['FlightDistance'] /(df['LifeTime'] * c)
    df['Delta_E'] = np.sqrt(np.square(np.sqrt(np.square(df['pt']) + np.square(df['pz'])) / df['beta_gamma']) + np.square(np.sqrt(np.square(df['pt']) + np.square(df['pz'])))) - df['E']
    df['Delta_M'] = np.square((np.sqrt(np.square(df['pt']) + np.square(df['pz']))) / df['beta_gamma']) - np.sqrt(np.square(df['E']) - np.square(np.sqrt(np.square(df['pt']) + np.square(df['pz']))))
    df['E0_ratio'] = df['E0'] / df['E']
    df['E1_ratio'] = df['E1'] / df['E']
    df['E2_ratio'] = df['E2'] / df['E']
    df['p0_pt_ratio'] = df['p0_pt'] / df['pt']
    df['p1_pt_ratio'] = df['p1_pt'] / df['pt']
    df['p2_pt_ratio'] = df['p2_pt'] / df['pt']
    df['eta_01'] = df['p0_eta'] - df['p1_eta']
    df['eta_02'] = df['p0_eta'] - df['p2_eta']
    df['eta_12'] = df['p1_eta'] - df['p2_eta']
    df['t_coll'] = (df['p0_pt'] + df['p1_pt'] + df['p2_pt']) / df['pt']
    df['flag_M'] = (np.fabs(np.sqrt(np.square(df['pt']) + np.square(df['pz'])) / df['beta_gamma'] - m_tau - 1.44)) < 17
    
    return df                     

In [11]:
# Add extra kinematic features:
train_with_extra_kinematic_features = add_extra_kinematic_features(train_with_extra_geometric_features)
test_with_extra_kinematic_features = add_extra_kinematic_features(test)

check_agreement_with_extra_kinematic_features = add_extra_kinematic_features(check_agreement_with_extra_geometric_features)
check_correlation_with_extra_kinematic_features = add_extra_kinematic_features(check_correlation_with_extra_geometric_features)

In [12]:
train_with_extra_kinematic_features = train_with_extra_kinematic_features[train_with_extra_kinematic_features.flag_M != False]

# 3. Train classifiers

In [13]:
params = {"objective": "binary:logistic",
          "eta": 0.08,
          "max_depth": 5,
          "min_child_weight": 3,
          "silent": 1,
          "subsample": 0.8,
          "colsample_bytree": 0.7,
          "seed": 1}
num_trees=200
gbm = xgb.train(params, xgb.DMatrix(train_with_extra_geometric_features[list1], train["signal"]), num_trees)

In [14]:
loss_funct = LogLossFunction()
ub = UGradientBoostingClassifier(loss=loss_funct,n_estimators=100, random_state=1,learning_rate=0.25,subsample=0.8)
ub.fit(train_with_extra_kinematic_features[list2],train_with_extra_kinematic_features["signal"])

UGradientBoostingClassifier(learning_rate=0.25,
              loss=LogLossFunction(regularization=5.0), max_depth=3,
              max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
              min_samples_split=2, n_estimators=100,
              random_state=<mtrand.RandomState object at 0x1137a2050>,
              splitter='best', subsample=0.8, train_features=None,
              update_tree=True)

# 4. Submit tests & record a submission file

In [15]:
agreement_probs = 0.18 * ub.predict_proba(check_agreement_with_extra_kinematic_features[list2])[:, 1] + 0.82 * gbm.predict(xgb.DMatrix(check_agreement_with_extra_geometric_features[list1]))

check_ks(agreement_probs)

('KS metric', 0.077056561802458956, True)

In [16]:
correlation_probs = 0.18 * ub.predict_proba(check_correlation_with_extra_kinematic_features[list2])[:, 1] + 0.82 * gbm.predict(xgb.DMatrix(check_correlation_with_extra_geometric_features[list1]))

cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])

print 'CvM metric', cvm, cvm < 0.002

CvM metric 0.00166004614494 True


In [17]:
train_eval = train_with_extra_kinematic_features

train_probs = 0.18 * ub.predict_proba(train_eval[list2])[:, 1] + 0.82 * gbm.predict(xgb.DMatrix(train_eval[list1]))

AUC = evaluation.roc_auc_truncated(train_eval['signal'], train_probs)
print ("AUC metric",AUC)

('AUC metric', 1.0)


In [18]:
test_probs = 0.18 * ub.predict_proba(test_with_extra_kinematic_features[list2])[:, 1] + 0.82 * gbm.predict(xgb.DMatrix(test_with_extra_geometric_features[list1]))

result = pd.DataFrame({'id': test.index, "prediction": test_probs})

In [19]:
result.to_csv('file_07_03_17_6.csv', index=False, sep=',')

#### The next part is just another algorithm (two xgboosts) which shown worse results, so there is no need to read it.

In [72]:
params = {'objective': 'binary:logistic',
          'eta': 0.05,
          'max_depth': 4,
          'scale_pos_weight': 5.,
          'silent': 1,
          'seed': 1}

num_trees1 = 200
num_trees2 = 100

# Weight for the first classifier:
w1 = 0.78

In [74]:
# Train the first (geometric) XGBoost classifier:
xgb1 = xgb.train(params, xgb.DMatrix(train_with_extra_geometric_features[list1], train['signal']), num_trees1)

In [75]:
# Train the second (kinematic) XGBoost classifier:
xgb2 = xgb.train(params, xgb.DMatrix(train_with_extra_kinematic_features[list2], train_with_extra_kinematic_features['signal']), num_trees2)

In [87]:
agreement_probs = 0.2 * xgb2.predict(xgb.DMatrix(check_agreement_with_extra_kinematic_features[list2])) + 0.8 * xgb1.predict(xgb.DMatrix(check_agreement_with_extra_geometric_features[list1]))

ks = evaluation.compute_ks(
    agreement_probs[check_agreement_with_extra_kinematic_features['signal'].values == 0],
    agreement_probs[check_agreement_with_extra_kinematic_features['signal'].values == 1],
    check_agreement_with_extra_kinematic_features[check_agreement_with_extra_kinematic_features['signal'] == 0]['weight'].values,
    check_agreement_with_extra_kinematic_features[check_agreement_with_extra_kinematic_features['signal'] == 1]['weight'].values)

print 'KS metric', ks, ks < 0.09

KS metric 0.0761471569486 True


In [88]:
correlation_probs = 0.2 * xgb2.predict(xgb.DMatrix(check_correlation_with_extra_kinematic_features[list2])) + 0.8 * xgb1.predict(xgb.DMatrix(check_correlation_with_extra_geometric_features[list1]))

cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])

print 'CvM metric', cvm, cvm < 0.002

CvM metric 0.000816380223291 True


In [89]:
train_eval = train_with_extra_kinematic_features[train_with_extra_kinematic_features['min_ANNmuon'] > 0.4]

print("calculating train probs having min_annmuon>0.4")

train_probs = 0.2 * xgb2.predict(xgb.DMatrix(train_eval[list2])) + 0.8 * xgb1.predict(xgb.DMatrix(train_eval[list1]))

calculating train probs having min_annmuon>0.4


In [90]:
AUC = evaluation.roc_auc_truncated(train_eval['signal'], train_probs)
print ("AUC metric",AUC)

('AUC metric', 0.99973080560499361)


In [91]:
test_probs = 0.2 * xgb2.predict(xgb.DMatrix(test_with_extra_kinematic_features[list2])) + 0.8 * xgb1.predict(xgb.DMatrix(test_with_extra_geometric_features[list1]))

result = pd.DataFrame({'id': test.index, "prediction": test_probs})

In [92]:
result.to_csv('file_07_03_17_8.csv', index=False, sep=',')